In [1]:
import scanpy as sc
import anndata
import pandas as pd
import pyranges as pr
import pybedtools
import sklearn
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm


/home/xetrin/miniconda3/envs/rna_atac_env/lib/python3.10/site-packages/sorted_nearest/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [11]:
print("Checkpoint TEST")

Checkpoint TEST


In [12]:
atac = sc.read_h5ad("/mnt/e/Projects/Brain/SEA_AD_analysis/results/aggregated_multiomic/aggregated_atac_data.h5ad")  # Peak x Cell matrix
print(atac)
sc.pp.normalize_total(atac, target_sum=1e4)
sc.pp.log1p(atac)
print(atac)
# Optional: binarize or apply TF-IDF normalization


AnnData object with n_obs × n_vars = 138118 × 218882
    obs: 'sample_id', 'Neurotypical reference', 'Donor ID', 'Organism', 'Brain Region', 'Sex', 'Gender', 'Age at Death', 'Race (choice=White)', 'Race (choice=Black/ African American)', 'Race (choice=Asian)', 'Race (choice=American Indian/ Alaska Native)', 'Race (choice=Native Hawaiian or Pacific Islander)', 'Race (choice=Unknown or unreported)', 'Race (choice=Other)', 'specify other race', 'Hispanic/Latino', 'Highest level of education', 'Years of education', 'PMI', 'Fresh Brain Weight', 'Brain pH', 'Overall AD neuropathological Change', 'Thal', 'Braak', 'CERAD score', 'Overall CAA Score', 'Highest Lewy Body Disease', 'Total Microinfarcts (not observed grossly)', 'Total microinfarcts in screening sections', 'Atherosclerosis', 'Arteriolosclerosis', 'LATE', 'Cognitive Status', 'Last CASI Score', 'Interval from last CASI in months', 'Last MMSE Score', 'Interval from last MMSE in months', 'Last MOCA Score', 'Interval from last MOCA in mo

In [13]:
print(atac.var.head())
peak_coords = atac.var.index.to_series().str.extract(r"(chr[\w]+):(\d+)-(\d+)")
peak_coords.columns = ["Chromosome", "Start", "End"]
peak_coords["Start"] = peak_coords["Start"].astype(int)
peak_coords["End"] = peak_coords["End"].astype(int)
peak_coords["peak_id"] = np.arange(len(peak_coords))

# Create PyRanges object
peaks = pr.PyRanges(peak_coords)
print(peaks)

                                          gene_ids feature_types
chr4:164130572-164131513  chr4:164130572-164131513         Peaks
chr4:16412872-16415576      chr4:16412872-16415576         Peaks
chr4:164116366-164116739  chr4:164116366-164116739         Peaks
chr4:164083047-164083256  chr4:164083047-164083256         Peaks
chr4:164056041-164057013  chr4:164056041-164057013         Peaks
+--------------+-----------+-----------+-----------+
| Chromosome   | Start     | End       | peak_id   |
| (category)   | (int64)   | (int64)   | (int64)   |
|--------------+-----------+-----------+-----------|
| chr1         | 43591097  | 43591599  | 164163    |
| chr1         | 43594973  | 43598008  | 164164    |
| chr1         | 43598680  | 43599017  | 164165    |
| chr1         | 43599915  | 43605694  | 164166    |
| ...          | ...       | ...       | ...       |
| chrY         | 13478174  | 13480771  | 73098     |
| chrY         | 13598891  | 13599575  | 73099     |
| chrY         | 9805339   

In [14]:
genes_df = pd.read_csv("/mnt/e/Projects/Brain/Data/SEAAD/gene_annotations_buffered_5kb.bed", sep="\t", names=["Chromosome", "Start", "End", "Gene"])
genes = pr.PyRanges(genes_df)
print(genes)
peak_to_gene = peaks.nearest(genes)
peak_to_gene = peak_to_gene.df
print(peak_to_gene)
#peak_to_gene = peaks.join(genes)
#print(peak_to_gene)

n_peaks = atac.shape[1]
genes = genes_df["Gene"].unique()
genes = pd.Series(genes).dropna().to_numpy()

+--------------+-----------+-----------+------------+
| Chromosome   | Start     | End       | Gene       |
| (category)   | (int64)   | (int64)   | (object)   |
|--------------+-----------+-----------+------------|
| chr1         | 3064168   | 3443621   | PRDM16     |
| chr1         | 5296928   | 5312394   | nan        |
| chr1         | 2398964   | 2418797   | PEX10      |
| chr1         | 5487978   | 5499674   | nan        |
| ...          | ...       | ...       | ...        |
| chrY         | 18686312  | 18729127  | nan        |
| chrY         | 18724882  | 18744197  | TTTY9A     |
| chrY         | 18475247  | 18489237  | nan        |
| chrY         | 18486740  | 18552698  | nan        |
+--------------+-----------+-----------+------------+
Unstranded PyRanges object has 78,649 rows and 4 columns from 24 chromosomes.
For printing, the PyRanges was sorted on Chromosome.
       Chromosome     Start       End  peak_id   Start_b     End_b     Gene  \
0            chr1  43591097  43591

In [15]:
gene_to_index = {gene: i for i, gene in enumerate(genes)}

# Filter valid rows first
valid_rows = peak_to_gene[
    peak_to_gene["Gene"].isin(gene_to_index) &
    peak_to_gene["peak_id"].notnull() &
    (peak_to_gene["peak_id"] < n_peaks)
].copy()

# Map genes to indices
valid_rows["gene_idx"] = valid_rows["Gene"].map(gene_to_index)
valid_rows["peak_idx"] = valid_rows["peak_id"].astype(int)

# Create sparse matrix directly
rows = valid_rows["peak_idx"].values
cols = valid_rows["gene_idx"].values
data = np.ones(len(rows), dtype=np.uint8)

peak_gene_map = sp.csr_matrix((data, (rows, cols)), shape=(n_peaks, len(genes)))

In [16]:
print("Checkpoint A")
# --- Ensure compatible sparse formats ---
atac_X = atac.X.tocsr() if not sp.isspmatrix_csr(atac.X) else atac.X
peak_gene_map_csc = peak_gene_map.tocsc()

print("Checkpoint B")
# --- Precompute normalization weights ---
peak_counts_per_gene = np.array(peak_gene_map_csc.sum(axis=0)).flatten()
peak_counts_per_gene[peak_counts_per_gene == 0] = 1
norm_diag = sp.diags(1 / peak_counts_per_gene)

print("Checkpoint C")
# --- Batch-wise matrix multiplication ---
batch_size = 5000  # Adjust lower if memory crashes again
n_cells = atac_X.shape[0]
result_chunks = []

print("Checkpoint D")
print("Performing matrix multiplication in batches...")
for start in tqdm(range(0, n_cells, batch_size)):
    end = min(start + batch_size, n_cells)
    batch = atac_X[start:end, :]
    chunk = batch @ peak_gene_map_csc   # cells x genes
    chunk = chunk @ norm_diag           # normalize
    result_chunks.append(chunk)

print("Checkpoint E")
# --- Combine into final access matrix ---
access_mat = sp.vstack(result_chunks)

print("Checkpoint F")

Checkpoint A
Checkpoint B
Checkpoint C
Checkpoint D
Performing matrix multiplication in batches...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:56<00:00,  2.02s/it]


Checkpoint E
Checkpoint F


In [17]:
#print("Checkpoint 0")

# Multiply: cells × peaks  @  peaks × genes  =  cells × genes
#access_mat = atac.X @ peak_gene_map  # keep sparse

#print("Checkpoint 1")

# Normalize by number of peaks per gene
#peak_counts_per_gene = np.array(peak_gene_map.sum(axis=0)).flatten()

#print("Checkpoint 2")

# Avoid division by zero
#peak_counts_per_gene[peak_counts_per_gene == 0] = 1

#print("Checkpoint 3")

# Construct a sparse diagonal matrix to divide columns
#norm_diag = sp.diags(1 / peak_counts_per_gene)  # genes × genes

#print("Checkpoint 4")

# This is safer than access_mat.multiply(...)
#access_mat = access_mat @ norm_diag  # normalized access matrix

#print("Checkpoint 5")

In [18]:
#access_mat = access_mat.toarray()
atac_gene = anndata.AnnData(X=access_mat)
atac_gene.obs_names = atac.obs_names
atac_gene.var_names = genes

In [19]:
print(atac_gene)
print(atac_gene.obs_names[1:5])
print(atac_gene.var_names[1:5])
test = atac_gene.X[1:15,1:5].toarray()
print(test)

AnnData object with n_obs × n_vars = 138118 × 41092
Index(['AGTTATGTCGAAGTAG-L8XR_210722_01_H07-1122543705',
       'CCTTCGGTCAGCAAGA-L8XR_210722_01_H07-1122543705',
       'TGAAGCAAGTAAGTGG-L8XR_210722_01_H07-1122543705',
       'ACCACATAGGCATTGT-L8XR_210722_01_H07-1122543705'],
      dtype='object', name='index')
Index(['RPL14P5', 'IL3RA', 'P2RY8', 'SLC25A6'], dtype='object')
[[0.         0.16088198 0.11897197 0.59485984]
 [0.         0.         0.         0.        ]
 [0.         0.17667045 0.80321226 2.14543366]
 [0.         0.         0.         0.79496056]
 [0.         0.         0.         0.48855361]
 [0.         0.04426422 0.         0.56558943]
 [0.         0.06854757 0.         1.09846878]
 [0.         0.         0.         0.94654435]
 [0.         0.         0.         0.        ]
 [0.         0.0734456  0.04451069 0.5096451 ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.0266467  0.         0.52678633]
 [0.    

In [20]:
print(atac_gene.var.index[1:5])
print(atac_gene.var.columns[1:5])

Index(['RPL14P5', 'IL3RA', 'P2RY8', 'SLC25A6'], dtype='object')
Index([], dtype='object')


In [21]:
atac_gene.var.index = atac_gene.var.index.astype(str)
atac_gene.var.columns = atac_gene.var.columns.astype(str)
atac_gene.write("/mnt/e/Projects/Brain/SEA_AD_analysis/results/aggregated_multiomic/aggregated_atac_geneLevel_data.h5ad")

In [ ]:

rna = sc.read_h5ad("/mnt/e/Projects/Brain/SEA_AD_analysis/results/aggregated_multiomic/aggregated_rna_data.h5ad")
print(rna)
sc.pp.filter_cells(rna, min_genes=200)
sc.pp.filter_genes(rna, min_cells=3)
sc.pp.normalize_total(rna, target_sum=1e4)
sc.pp.log1p(rna)
sc.pp.highly_variable_genes(rna, n_top_genes=2000)
rna = rna[:, rna.var.highly_variable]
sc.pp.scale(rna)
sc.tl.pca(rna)
print(rna)


AnnData object with n_obs × n_vars = 138118 × 36601
    obs: 'sample_id', 'Neurotypical reference', 'Donor ID', 'Organism', 'Brain Region', 'Sex', 'Gender', 'Age at Death', 'Race (choice=White)', 'Race (choice=Black/ African American)', 'Race (choice=Asian)', 'Race (choice=American Indian/ Alaska Native)', 'Race (choice=Native Hawaiian or Pacific Islander)', 'Race (choice=Unknown or unreported)', 'Race (choice=Other)', 'specify other race', 'Hispanic/Latino', 'Highest level of education', 'Years of education', 'PMI', 'Fresh Brain Weight', 'Brain pH', 'Overall AD neuropathological Change', 'Thal', 'Braak', 'CERAD score', 'Overall CAA Score', 'Highest Lewy Body Disease', 'Total Microinfarcts (not observed grossly)', 'Total microinfarcts in screening sections', 'Atherosclerosis', 'Arteriolosclerosis', 'LATE', 'Cognitive Status', 'Last CASI Score', 'Interval from last CASI in months', 'Last MMSE Score', 'Interval from last MMSE in months', 'Last MOCA Score', 'Interval from last MOCA in mon

In [ ]:
from sklearn.cross_decomposition import CCA

# Reduce dimensionality
sc.pp.pca(rna, n_comps=30)
sc.pp.pca(atac_gene, n_comps=30)

cca = CCA(n_components=20)
rna_cca, atac_cca = cca.fit_transform(rna.obsm["X_pca"], atac_gene.obsm["X_pca"])

In [ ]:
shared_embedding = np.concatenate([rna_cca, atac_cca])

In [ ]:
# Combine
joint = anndata.AnnData(X=shared_embedding)
joint.obs["modality"] = ["rna"] * rna.n_obs + ["atac"] * atac.n_obs

sc.pp.neighbors(joint, n_neighbors=15)
sc.tl.umap(joint)
sc.tl.leiden(joint)
sc.pl.umap(joint, color=["modality", "leiden"])

In [ ]:
joint.obs["Subclass"] = (
    list(rna.obs["Subclass"] if "Subclass" in rna.obs else ["NA"] * rna.n_obs)
    + list(atac.obs["Subclass"])
)

In [ ]:
print(joint.obs.columns)

In [ ]:
sc.pl.umap(joint, color=["Subclass"])